In [ ]:
import utils

In [ ]:
from ultralytics import YOLO

# Load a COCO-pretrained YOLO11n-seg model
model = YOLO("yolo11n-seg.pt")

# Train the model for 100 epochs
results = model.train(data="datasets/goat_yolo_v11/data.yaml", epochs=100, imgsz=640, name="torso_yolov11n_2", save=True)

In [ ]:
from ultralytics import YOLO

model = YOLO("./best.pt")

In [ ]:


path = "datasets/goat_yolo_v11/test/images/199_jpg.rf.b5c8920899ddd54b9ab73eb698b3ec2b.jpg"
path = "datasets/goat_yolo_v11/test/images/1303_jpg.rf.5d2842ed5368d6db2d38ded4e8948237.jpg"
path = "datasets/goat_yolo_v11/test/images/106_jpg.rf.20ebe916320dc0f47302993b68e3662e.jpg"
path = "datasets/goat_yolo_v11/test/images/179_jpg.rf.130da7b6ed3713c19597656741428f5e.jpg"
path = "datasets/goat_yolo_v11/test/images/1294_jpg.rf.22f6fc35b32c014b57856baf0343fd97.jpg"
path = "datasets/erste_messung"


# Predict with the model
results = model(path)  # predict on an image

for result in results:
    utils.plot(result)



In [ ]:
from ultralytics import SAM
import glob
import cv2

# Load a model
model = SAM("sam2.1_b.pt")

# Display model information (optional)
model.info()

for img in glob.glob("datasets/erste_messung/*.JPG"):
    # Run inference with bboxes prompt
    img = cv2.imread(img)
    height, width, _ = img.shape
    print(height, width)
    results = model(img, points=[width/2, height/2])
    utils.plot(results[0])

In [ ]:
model = SAM("sam2.1_b.pt")
model.export(format="onnx")

In [ ]:
from ultralytics import YOLO

model = YOLO("./best.pt")
model.export(format="tfjs") 



In [ ]:
# Load the exported TF.js model
tfjs_model = YOLO("./best_web_model")

# Run inference
results = tfjs_model("datasets/goat_yolo_v11/test/images/1294_jpg.rf.22f6fc35b32c014b57856baf0343fd97.jpg")

In [ ]:
path = "datasets/erste_messung/IMG_1600_Twentyone.JPG"

# Predict with the model
results = model(path)  # predict on an image

for result in results:
    mask = result.masks.data.cpu()[0]
    body_length, shoulder_height, sacrum_height = utils.body_measurement(mask.numpy())
    distance = 1.64
    print(body_length, shoulder_height, sacrum_height)
    body_length = utils.pixels_to_cm(scale_to_width(body_length), distance)
    shoulder_height = utils.pixels_to_cm(scale_to_height(shoulder_height), distance)
    sacrum_height = utils.pixels_to_cm(scale_to_height(sacrum_height), distance)
    print("body_length cm:", body_length)
    print("shoulder height cm: ", shoulder_height)
    print("sacrum height cm:", sacrum_height)

    

In [ ]:
from transformers import pipeline
from PIL import Image

path = "datasets/erste_messung/IMG_1600_Twentyone.JPG"
image = Image.open(path)

estimator = pipeline(task="depth-estimation", model="Intel/zoedepth-nyu-kitti")
result = estimator(image)
result

In [ ]:
result["depth"]

In [ ]:
import matplotlib.pyplot as plt

plt.imshow(result["depth"])
pd = result["predicted_depth"]
pd = pd.numpy()
print(pd.shape)
print(pd[int(pd.shape[0]/2), int(pd.shape[1]/2)])
plt.plot(int(pd.shape[1]/2),int(pd.shape[0]/2), 'ro')
plt.plot(1300, 1500, 'ro')
print(pd[1500, 1300])

In [ ]:
from PIL import Image
import torch
from transformers import DepthProImageProcessorFast, DepthProForDepthEstimation

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

path = "datasets/erste_messung/IMG_1600_Twentyone.JPG"
image = Image.open(path)

image_processor = DepthProImageProcessorFast.from_pretrained("apple/DepthPro-hf")
model = DepthProForDepthEstimation.from_pretrained("apple/DepthPro-hf").to(device)

inputs = image_processor(images=image, return_tensors="pt").to(device)

with torch.no_grad():
    outputs = model(**inputs)

post_processed_output = image_processor.post_process_depth_estimation(
    outputs, target_sizes=[(image.height, image.width)],
)

field_of_view = post_processed_output[0]["field_of_view"]
focal_length = post_processed_output[0]["focal_length"]
depth = post_processed_output[0]["predicted_depth"]
depth = (depth - depth.min()) / (depth.max() - depth.min())
depth = depth * 255.
depth = depth.detach().cpu().numpy()
depth = Image.fromarray(depth.astype("uint8"))

In [ ]:
plt.imshow(depth)
depth = post_processed_output[0]["predicted_depth"]
depth[1300, 1500]

In [ ]:
from transformers import pipeline
from PIL import Image

pipe = pipeline(task="depth-estimation", model="depth-anything/Depth-Anything-V2-Small")
path = "datasets/erste_messung/IMG_1600_Twentyone.JPG"
image = Image.open(path)
depth = pipe(image)["depth"]

In [ ]:
plt.imshow(depth)
depth_map = post_processed_output[0]["predicted_depth"]
print(depth_map.shape)
print(depth_map[int(depth_map.shape[0]/2), int(depth_map.shape[1]/2)])
plt.plot(int(depth_map.shape[0]/2), int(depth_map.shape[1]/2), 'ro')

In [ ]:
from transformers import pipeline
from PIL import Image

pipe = pipeline(task="depth-estimation", model="onnx-community/depth-anything-v2-small")
path = "datasets/erste_messung/IMG_1600_Twentyone.JPG"
image = Image.open(path)
depth = pipe(image)["depth"]